In [1]:
import numpy as np
import pandas as pd
import pickle
from pathlib import Path
from kuma_utils.utils import sigmoid

In [2]:
model_path = Path('results/aug_04_ds2/')

In [3]:
submission_path = model_path/'predictions.npy'
predictions = sigmoid(np.load(submission_path))
test = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/sample_submission.csv')
public748 = pd.read_csv('input/public748.csv')

In [4]:
# test['target'] = predictions.reshape(-1) * 0.2 + public748['target'].values * 0.8
test['target'] = predictions.reshape(-1)
# test['target'] = predictions[0]

In [5]:
test

,id,target
0,00054c878,0.778312
1,0007285a3,0.971773
2,00076c5a6,0.815042
3,001349290,0.786822
4,001a52e92,0.822169
...,...,...
7970,ffbce04ef,0.999686
7971,ffc2d976b,0.775661
7972,ffc905909,0.776613
7973,ffe276f3e,0.788620


In [6]:
test.to_csv(model_path/'submission.csv', index=False)
# test.to_csv(model_path/'blend_submission.csv', index=False)

# Split

In [35]:

# from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
# df = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/v1.csv')
# splitter = MultilabelStratifiedKFold(n_splits=5, shuffle=True)
# y = pd.get_dummies(pd.cut(df['signal_depth'], [0, 20, 40, 60, 80, 100, 1000]))
# for train_idx, valid_idx in list(splitter.split(df, y)):
#     train_fold = df.iloc[train_idx]
#     valid_fold = df.iloc[valid_idx]
#     print(train_fold['target'].mean(), valid_fold['target'].mean())
#     display(y.iloc[train_idx].mean(0))
#     display(y.iloc[valid_idx].mean(0))